In [12]:
import pickle
import numpy as np
import pandas as pd
import math
import sys, os
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [14]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [19]:
from selector import *
from vizualizing_data import *
from outlier import Outlier

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
pd.set_option('display.max_columns', 100)

In [24]:
df = pd.read_csv("../data/clean_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146887 entries, 0 to 146886
Data columns (total 56 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Unnamed: 0                        146887 non-null  int64  
 1   Bearer Id                         146887 non-null  float64
 2   Start                             146887 non-null  object 
 3   Start ms                          146887 non-null  float64
 4   End                               146887 non-null  object 
 5   End ms                            146887 non-null  float64
 6   Dur. (ms)                         146887 non-null  float64
 7   IMSI                              146887 non-null  float64
 8   MSISDN/Number                     146887 non-null  float64
 9   IMEI                              146887 non-null  float64
 10  Last Location Name                146887 non-null  object 
 11  Avg RTT DL (ms)                   146887 non-null  f

# User Experience Analysis

In [25]:
user_experience_df = df[[
    'MSISDN/Number',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)',
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Handset Type']]

In [26]:
# caluclate the totals
user_experience_df['Total Avg RTT (ms)'] = user_experience_df['Avg RTT DL (ms)'] + user_experience_df['Avg RTT UL (ms)']
user_experience_df['Total Avg Bearer TP (kbps)'] = user_experience_df['Avg Bearer TP DL (kbps)'] + user_experience_df['Avg Bearer TP UL (kbps)']
user_experience_df['Total TCP Retrans. Vol (Bytes)'] = user_experience_df['TCP DL Retrans. Vol (Bytes)'] + user_experience_df['TCP UL Retrans. Vol (Bytes)']
user_experience_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146887 entries, 0 to 146886
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   MSISDN/Number                   146887 non-null  float64
 1   Avg RTT DL (ms)                 146887 non-null  float64
 2   Avg RTT UL (ms)                 146887 non-null  float64
 3   Avg Bearer TP DL (kbps)         146887 non-null  float64
 4   Avg Bearer TP UL (kbps)         146887 non-null  float64
 5   TCP DL Retrans. Vol (Bytes)     146887 non-null  float64
 6   TCP UL Retrans. Vol (Bytes)     146887 non-null  float64
 7   Handset Type                    146887 non-null  object 
 8   Total Avg RTT (ms)              146887 non-null  float64
 9   Total Avg Bearer TP (kbps)      146887 non-null  float64
 10  Total TCP Retrans. Vol (Bytes)  146887 non-null  float64
dtypes: float64(10), object(1)
memory usage: 12.3+ MB


In [27]:
user_experience_df.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,3.366496e+10,42.0,5.0,23.0,44.0,19520.0,7230.0,Samsung Galaxy A5 Sm-A520F,47.0,67.0,26750.0
1,3.368185e+10,65.0,5.0,16.0,26.0,19520.0,7230.0,Samsung Galaxy J5 (Sm-J530),70.0,42.0,26750.0
2,3.373168e+10,65.0,5.0,6.0,9.0,19520.0,7230.0,Samsung Galaxy A8 (2018),70.0,15.0,26750.0
3,3.373168e+10,65.0,5.0,44.0,44.0,19520.0,7230.0,undefined,70.0,88.0,26750.0
4,3.369980e+10,65.0,5.0,6.0,9.0,19520.0,7230.0,Samsung Sm-G390F,70.0,15.0,26750.0


In [28]:
# select the necessary columns
user_experience_df = user_experience_df[['MSISDN/Number', 'Handset Type', 'Total Avg RTT (ms)',
    'Total Avg Bearer TP (kbps)', 'Total TCP Retrans. Vol (Bytes)']]
user_experience_df.head()

,MSISDN/Number,Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,3.366496e+10,Samsung Galaxy A5 Sm-A520F,47.0,67.0,26750.0
1,3.368185e+10,Samsung Galaxy J5 (Sm-J530),70.0,42.0,26750.0
2,3.373168e+10,Samsung Galaxy A8 (2018),70.0,15.0,26750.0
3,3.373168e+10,undefined,70.0,88.0,26750.0
4,3.369980e+10,Samsung Sm-G390F,70.0,15.0,26750.0


In [29]:
# aggregating user experience metrics per user
user_experience_df1 = user_experience_df.groupby(
    'MSISDN/Number').agg({
        'Total Avg RTT (ms)': 'sum',
        'Total Avg Bearer TP (kbps)': 'sum',
        'Total TCP Retrans. Vol (Bytes)': 'sum',
        'Handset Type': [lambda x: x.mode()[0]]})
user_experience_df1.head()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
,sum,sum,sum,<lambda>
MSISDN/Number,,,,
3.360304e+10,47046.5,7624337.0,1.424016e+09,undefined
3.360304e+10,40.0,102.0,7.488492e+06,Apple iPhone Xr (A2105)
3.360304e+10,35.0,106.0,7.824628e+06,Samsung Galaxy J3 (Sm-J330)
3.360304e+10,129.0,1529.0,1.293200e+04,Apple iPhone 6 Plus (A1524)
3.360304e+10,104.5,52836.0,2.772426e+06,Samsung Galaxy S8 (Sm-G950F)


In [30]:
user_experience_df = pd.DataFrame(columns=[
    "Total Avg RTT (ms)",
    "Total Avg Bearer TP (kbps)",
    "Total TCP Retrans. Vol (Bytes)",
    "Handset Type"])

user_experience_df["Total Avg RTT (ms)"] = user_experience_df1["Total Avg RTT (ms)"]['sum']
user_experience_df["Total Avg Bearer TP (kbps)"] = user_experience_df1["Total Avg Bearer TP (kbps)"]['sum']
user_experience_df["Total TCP Retrans. Vol (Bytes)"] = user_experience_df1["Total TCP Retrans. Vol (Bytes)"]['sum']
user_experience_df["Handset Type"] = user_experience_df1["Handset Type"]['<lambda>']
user_experience_df.head()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
3.360304e+10,47046.5,7624337.0,1.424016e+09,undefined
3.360304e+10,40.0,102.0,7.488492e+06,Apple iPhone Xr (A2105)
3.360304e+10,35.0,106.0,7.824628e+06,Samsung Galaxy J3 (Sm-J330)
3.360304e+10,129.0,1529.0,1.293200e+04,Apple iPhone 6 Plus (A1524)
3.360304e+10,104.5,52836.0,2.772426e+06,Samsung Galaxy S8 (Sm-G950F)


#### Top 10 values of the experience metrics

In [31]:
# top 10 rtt values in the dataset

rtt = user_experience_df.sort_values('Total Avg RTT (ms)', ascending=False)
top_10 = rtt.head(10)['Total Avg RTT (ms)']
bottom_10 = rtt.tail(10)['Total Avg RTT (ms)']
most_10 = user_experience_df['Total Avg RTT (ms)'].value_counts().head(10)

In [32]:
plotly_multi_hist([top_10, bottom_10, most_10], 1, 3,
    "RTT values in the dataset", ['Top 10', 'Bottom 10', 'Most 10'])

NameError: name 'plotly_multi_hist' is not defined